# Imports

In [1]:
%matplotlib qt
import generate_data as gd
import utils_V2 as u2
import numpy as np
import hyperspy.api as hs
import matplotlib.pyplot as plt

# Parameters

In [2]:
# To save the dataset
filename = "aspim036_N150_2ptcls_brstlg"

# Continuum X-rays parameters
# They were determine by fitting experimental data from 0.6 to 18 keV. Since low energies were incorporated, the model is only effective and not quantitative. 
c0  =  3.943136127751902
c1  =  3.9446849862408535 
c2  =  0.027663073842682524
b0  =  0.1414560446115408
b1  =  -0.1057210517202927
b2  =  0.026461615841445782

scale = 1.0
method = "simplified2"

# Average number of counts in one spectrum of the artificial data
N = 150 

# Creation of the pure spectra of the different phases.
phase1 = u2.Gaussians()
# Gaussians corresponding to elements
phase1.create_spectrum({"O-K":1.0,"Si-K":0.61,"Al-K":0.07,"Mg-K":0.51,"Ca-K":0.02,"Ca-L":0.04,"Sm-M":0.02,"Sm-L":0.0025,"Fe-K":0.028,"Nd-L":0.002,"Nd-M":0.002,"Lu-L":0.003,"Lu-M":0.003,"Hf-L":0.003,"Hf-M":0.003,"Cu-K":0.01,"Cu-L":0.01})
# Continuum X-rays
phase1.add_bremsstrahlung(method,scale,b0,b1,b2,c0,c1,c2)

phase2 = u2.Gaussians()
phase2.create_spectrum({"O-K":0.54,"Mg-K":1.0,"Fe-K":0.15,"Fe-L":0.06,"Cu-K":0.025,"Cu-L":0.038,"U-M":0.0052,"Nd-L":0.004,"Nd-M":0.002,"Ga-K":0.003,"Lu-L":0.003,"Lu-M":0.003})
phase2.add_bremsstrahlung(method,scale,b0,b1,b2,c0,c1,c2)

phase3 = u2.Gaussians()
phase3.create_spectrum({"O-K":1.0,"Si-K":0.12,"Al-K":0.18,"Ca-K":0.47,"Ca-L":0.47,"Sm-M":0.04,"Sm-L":0.04,"Fe-K":0.004,"Nd-L":0.008,"Nd-M":0.008,"Hf-L":0.004,"Hf-M":0.004,"Cu-K":0.01,"Cu-L":0.01})
phase3.add_bremsstrahlung(method,scale,b0,b1,b2,c0,c1,c2)

# Objects needed for the creation of data
# list of spectra
phases = [phase1.y,phase2.y,phase3.y] 
# list of densities which will give different total number of events per spectra
densities=[1.0,1.33,1.25]

# Artificial data creation

Note : Each function of ArtificialSpim updates the current object and adds on top. It is usually better to re-create a new object to change the artificial data settings.
Spim corresponds to spectrum image.

In [11]:
# Creation of an ArtificialSpim instance
spim=gd.AritificialSpim(phases,densities,(80,80))

In [12]:
# We add two particles belonging to two different phases to the data
spim.sphere((25,30),3.5,3.5,0.0,0.5,1)
spim.sphere((55,30),3.5,3.5,0.0,0.5,2)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
# Generates a noiseless version of the dataset
spim.generate_spim_deterministic()

array([[24.60468773, 24.60468773, 24.60468773, ..., 24.60468773,
        24.60468773, 24.60468773],
       [24.60468773, 24.60468773, 24.60468773, ..., 24.60468773,
        24.60468773, 24.60468773],
       [24.60468773, 24.60468773, 24.60468773, ..., 24.60468773,
        24.60468773, 24.60468773],
       ...,
       [24.60468773, 24.60468773, 24.60468773, ..., 24.60468773,
        24.60468773, 24.60468773],
       [24.60468773, 24.60468773, 24.60468773, ..., 24.60468773,
        24.60468773, 24.60468773],
       [24.60468773, 24.60468773, 24.60468773, ..., 24.60468773,
        24.60468773, 24.60468773]])

In [14]:
# Generate noisy spectra based on a Poisson distribution with average N
spim.generate_spim_stochastic(N)

In [7]:
# To save the data and the ground truth under similar names
spim.save(filename)

# Data visualisation

In [8]:
# To visualise the pure spectrum of the first phase
x = u2.Gaussians().x
plt.plot(x,phases[0],"g-",label="Matrix",linewidth=2)
plt.plot(x,u2.Distributions.simplified_brstlg_2(x,b0,b1,b2,c0,c1,c2),"r--",label="Continuum",linewidth=2)
plt.xlabel("Energy [keV]")
plt.ylabel("Intensity")
plt.rcParams.update({'font.size': 22})
plt.legend()
plt.tight_layout()

## Interactive visualization of the created datacube

3 windows will pop : 

1) Map of the sum of counts per pixel - A green square will be plotted as well at the top left

2) Spectrum of the top left corner pixel (hyperspy built-in)

3) Sum of spectra of the green area in 1)

It possible to move and reshape the green area in 1) to explore the data

In [15]:
signal=hs.signals.Signal1D(spim.stochastic_spim)
roi=hs.roi.RectangularROI(12,12,24,24)
signal.plot()
spim_ROI=roi.interactive(signal)
    
sum_ROI=hs.interactive(spim_ROI.sum,
               event=spim_ROI.axes_manager.events.any_axis_changed,
               recompute_out_event=None)
sum_ROI.plot()

using interactive roi
removing widget
